<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Approximate-posterior-variance" data-toc-modified-id="Approximate-posterior-variance-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Approximate posterior variance</a></span></li><li><span><a href="#KL-terms" data-toc-modified-id="KL-terms-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>KL terms</a></span><ul class="toc-item"><li><span><a href="#inner-product-term" data-toc-modified-id="inner-product-term-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>inner product term</a></span></li><li><span><a href="#trace-term" data-toc-modified-id="trace-term-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>trace term</a></span></li><li><span><a href="#log-det-term" data-toc-modified-id="log-det-term-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>log-det term</a></span></li></ul></li><li><span><a href="#full-KL" data-toc-modified-id="full-KL-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>full KL</a></span></li></ul></div>

In [1]:
%cd ../gpsig

/Users/maudlemercier/Desktop/GPSig/gpsig


In [2]:
import sys
sys.path.append('..') # add to path parent dir of gpsig

# numerics
import numpy as np

# signatures
import esig
esig.is_library_loaded()
import gpsig

import gpflow as gp
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import iisignature
import time

from keras import backend as K


The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The savefig.jpeg_quality rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The keymap.all_axes rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The animation.avconv_path rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The animation.avconv_args rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
Using TensorFlow backend.


**Define problem dimensions**

In [3]:
num_examples = 3
len_examples = 20
num_features = 3

num_tensors = 2
num_levels = 4 

**Define kernel to access its fast methods**

In [4]:
input_dim = num_features * len_examples
kern = gpsig.kernels.SignatureLinear(input_dim, num_features, num_levels, order=num_levels, normalization=False)

**Create paths and their signatures**

In [5]:
X = 0.1*np.cumsum(np.random.randn(num_examples, len_examples, num_features),axis=1)

sigs = np.asarray([esig.tosig.stream2sig(x, num_levels) for x in X])

**Create sparse tensors for the ariational means and covariances**

In [19]:
len_tensors = int(num_levels*(num_levels+1)/2)

# create R variational means sparse tensors
means = 3*np.random.randn(len_tensors,num_tensors,num_features)**2

# create R variational covariance (diagonal) sparse tensors
Lambdas = 3*np.random.randn(len_tensors,num_tensors,num_features)**2

**Flatten sparse tensors to compute ground truth naively**

In [20]:
def flatten(Z):
    flat = [np.ones((num_tensors,1))]
    k = 0
    for m in range(1, num_levels+1):
        Zm = Z[k]
        k += 1
        for i in range(1, m):
            Zm = (Zm[..., None] * Z[k, :, None, :]).reshape([num_tensors, -1])
            k += 1
        flat.append(Zm)
    flat = np.concatenate(flat, axis=1)
    return flat

means_flat = flatten(means)
Lambdas_flat = flatten(Lambdas)

# Approximate posterior variance

**Compute ground truth naively**

In [21]:
naive = ((1.-Lambdas_flat[:,None,:])*sigs[None,:,:]) @ sigs.T
naive = np.stack([np.diag(naive[i]) for i in range(num_tensors)]).T

**Compute with fast algo**

In [22]:
res = kern.compute_K_diag_rescaled(Lambdas,X.reshape([num_examples, -1]))

In [23]:
res

array([[  0.02126579,  -0.8112408 ],
       [  0.03677728,  -0.90730117],
       [  0.10632   , -10.71846802]])

In [24]:
abs(res-naive)<1e-6

array([[ True,  True],
       [ True,  True],
       [ True,  True]])

# KL terms

## inner product term

**Compute ground truth naively**

In [25]:
means_flat.shape

(2, 121)

In [26]:
naive = np.sum(means_flat**2, axis=1)

**Compute with fast algo**

In [27]:
res = kern.compute_norms_tens(means)

In [28]:
res

array([2471252.38470478, 1217013.29614907])

In [29]:
abs(res-naive)<1e-6

array([ True,  True])

## trace term

**Compute ground truth naively**

In [30]:
naive = np.sum(Lambdas_flat, axis=1)

**Compute with fast algo**

In [31]:
res = kern.compute_sums_tens(Lambdas)

In [32]:
res

array([ 790.57475561, 2297.2351363 ])

In [33]:
abs(res-naive)<1e-6

array([ True,  True])

## log-det term

**Compute ground truth naively**

In [34]:
naive = np.sum(np.log(Lambdas_flat), axis=1)

**Compute with fast algo**

In [35]:
res = kern.compute_logs_tens(Lambdas)

In [36]:
res

array([-323.48873562,   18.62772995])

In [37]:
abs(res-naive)<1e-6

array([ True,  True])

# full KL

In [38]:
bias = 3*np.random.randn(1,num_tensors)
first_var = 3*np.random.randn(1,num_tensors)**2

def flatten(Z,bias):
    flat = [bias.T]
    k = 0
    for m in range(1, num_levels+1):
        Zm = Z[k]
        k += 1
        for i in range(1, m):
            Zm = (Zm[..., None] * Z[k, :, None, :]).reshape([num_tensors, -1])
            k += 1
        flat.append(Zm)
    flat = np.concatenate(flat, axis=1)
    return flat

means_flat = flatten(means,bias)
Lambdas_flat = flatten(Lambdas,first_var)

In [39]:
from gpflow.kullback_leiblers import gauss_kl

with tf.Session(graph=tf.Graph()) as sess:
    
    # likelihood
    num_latent = num_tensors
    lik = gp.likelihoods.MultiClass(num_latent)

    # inducing var
    num_inducing = np.sum([ num_features**i for i in range(num_levels+1)])
    feat = gpsig.inducing_variables_vosf.UntruncInducingOrthogonalTensors(input_dim=input_dim, d = num_features, M = num_inducing) 

    # kernel
    k = gpsig.kernels_pde.UntruncSignatureKernel(input_dim, num_features=num_features, order=0, implementation='cython')
    
    # model
    X_ = K.placeholder(shape=(num_examples, input_dim), dtype=gp.settings.float_type)
    y_ = K.placeholder(shape=(num_examples, 1), dtype=gp.settings.float_type)
    m = gpsig.models.SVGP(X_, y_ , kern=k, feat=feat, likelihood=lik, num_latent=num_latent, q_diag = True,
                              minibatch_size=None, whiten=True, fast_algo=True, q_mu = means, q_sqrt = Lambdas, bias=bias,first_var=first_var)
    
    # commented in build_likelihood the non-KL term to check
    likelihood = m.likelihood_tensor
    res = sess.run(likelihood, feed_dict={X_:X.reshape([num_examples, -1]), y_:np.random.randn(num_examples,1) })
    
    naive = gauss_kl(tf.constant(means_flat.T),tf.constant(np.sqrt(Lambdas_flat.T)))
    naive = sess.run(naive)

In [40]:
res

1845718.9019086019

In [41]:
naive

1845718.9019086016